In [1]:
from collections import namedtuple
from itertools import product
from enum import Enum, auto

    
C2 = namedtuple("C2", "x y")
C3 = namedtuple("C3", "x y z")


class Color(Enum):
    WHITE = 1
    BLUE = 2
    RED = 3
    GREEN = 4  # Initial piece


class Shape(Enum):
    LONG4 = C3(1, 1, 4)
    RECTANGLE3 = C3(2, 1, 3)
    RECTANGLE2 = C3(3, 1, 2)
    SQUARE2 = C3(2, 1, 2)
    SHORT2 = C3(1, 1, 2)
    LONG1 = C3(4, 1, 1)
    RECTANGLE1 = C3(3, 2, 1)
    SQUARE1 = C3(2, 2, 1)
    SHORT1 = C3(2, 1, 1)
    

# Piece is oriented so that colored side is always up.
# Length is always >= to width
Piece = namedtuple("Piece", "color shape")
PIECES = frozenset(
    Piece(color, shape) for color, shape in product([Color.WHITE, Color.BLUE, Color.RED], Shape)
)


def rotate(coords: C3) -> C3:
    return C3(coords.y, coords.x, coords.z)


class FirstPos(Enum):
    """
    A is the longest side (3), B the second one (2) and C she shortest (1).
    Order is x (longest side of the board), y (shortest side), z (height)
    Second argument is rotation
    """
    ABC = (Shape.RECTANGLE1, False)
    ACB = (Shape.RECTANGLE2, False)
    BAC = (Shape.RECTANGLE1, True)
    BCA = (Shape.RECTANGLE3, False)
    CAB = (Shape.RECTANGLE2, True)
    CBA = (Shape.RECTANGLE3, True)


class Card(Enum):
    WHITE = frozenset(p for p in PIECES if p.color == Color.WHITE)
    BLUE = frozenset(p for p in PIECES if p.color == Color.BLUE)
    RED = frozenset(p for p in PIECES if p.color == Color.RED)
    LONG = frozenset(p for p in PIECES if sorted(p.shape.value)[:2] == [1, 1])
    SQUARE = frozenset(p for p in PIECES if sorted(p.shape.value) == [1, 2, 2])
    RECTANGLE = frozenset(p for p in PIECES if sorted(p.shape.value) == [1, 2, 3])


INITIAL_DECK = [Card.SQUARE, Card.WHITE, Card.LONG, Card.RECTANGLE, Card.BLUE,
                Card.RED, Card.WHITE, Card.RECTANGLE, Card.WHITE, Card.BLUE,
                Card.RED, Card.SQUARE, Card.LONG, Card.RECTANGLE, Card.BLUE,
                Card.LONG, Card.LONG, Card.BLUE, Card.SQUARE, Card.RED, Card.LONG,
                Card.BLUE, Card.RED, Card.LONG, Card.RECTANGLE, Card.SQUARE,
                Card.WHITE, Card.BLUE, Card.WHITE, Card.WHITE, Card.RED, Card.RED,
                Card.RECTANGLE, Card.RECTANGLE, Card.SQUARE, Card.SQUARE]


CARDS_NUM = len(INITIAL_DECK)


BOARD = C2(6, 4)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    
def draw(pieces):
    
    # prepare some coordinates
    shape = (BOARD.x, BOARD.y, 7)
    x, y, z = np.indices(shape)
    colors = np.empty(shape, dtype=object)

    voxels = (x == -1)
    
    base = (z == 0)
    voxels |= base
    colors[base] = 'k'
    
    for (pos, shape), color in pieces.items():
        voxel = ((pos.x <= x) & (x < pos.x + shape.x) &
                 (pos.y <= y) & (y < pos.y + shape.y) &
                 (pos.z + 1 <= z) & (z < pos.z + shape.z + 1))
        colors[voxel] = color.name.lower()
        voxels |= voxel

    # and plot everything
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_aspect('equal')
    ax.voxels(voxels, facecolors=colors, edgecolor='k')
    set_axes_equal(ax)
    plt.show()



In [3]:
from typing import Sequence
import numpy as np

class InvalidMove(ValueError):pass
class PieceNotAvaliable(InvalidMove): pass
class PieceNotIncludedInCare(InvalidMove): pass
class OutOfBounds(InvalidMove): pass
class PieceNotOnOtherPieces(InvalidMove): pass
class PieceNextToSimilarPiece(InvalidMove): pass
class PieceNotNextToAnotherPiece(InvalidMove): pass


class Game:
    def __init__(
        self,
        turn: int,
        deck: Sequence[Card],
        used: Sequence[Card],
        board_cubes: np.array,
        board_pieces,
        first: FirstPos,
    ):
        assert turn < CARDS_NUM
        self.turn = turn
        self.deck = deck
        self.used = used
        self.board_cubes = board_cubes
        self.board_pieces = board_pieces
        self.first = first

    @classmethod
    def new(cls, first: FirstPos, cards_order: Sequence[int] = None):
        first_shape, rotated = first.value
        first_shape = first_shape.value
        if rotated:
            first_shape = rotate(first_shape)
        
        cards_order = cards_order or range(CARDS_NUM)
        assert len(cards_order) == CARDS_NUM and set(cards_order) == set(range(CARDS_NUM))
        deck = [INITIAL_DECK[i] for i in cards_order]
        board_cubes = np.zeros(BOARD, dtype=int)
        board_cubes[:first_shape.x,:first_shape.y] = first_shape.z
        board_pieces = {(C3(0, 0, 0), first_shape): Color.GREEN}
        return cls(turn=0, deck=deck, used=[], board_cubes=board_cubes, board_pieces=board_pieces, first=first)
        
    @property
    def card(self):
        return self.deck[self.turn]
    
    def draw(self):
        draw(pieces=self.board_pieces)
        
    def move(self, piece: Piece, rotated: bool, pos:C2):
        return Move(game=self, piece=piece, rotated=rotated, pos=pos)
    
    def possible_moves(self):
        for piece in PIECES - frozenset(self.used):
            for rotated in random.sample((True, False), 2):
                # Let's optimize this later
                for x in random.sample(range(BOARD.x), BOARD.x):
                    for y in random.sampl(range(BOARD.y), BOARD.y):
                        move = self.move(piece=piece, rotated=rotated, pos=C2(x, y))
                        try:
                            move.check()
                        except InvalidMove:
                            pass
                        else:
                            yield move
    
class Move:
    def __init__(self, game:Game, piece: Piece, rotated: bool, pos:C2):
        self.game = game
        self.piece = piece
        self.rotated = rotated
        self.pos = pos
        self.pos3 = C3(*pos, self.game.board_cubes[pos])
        
        self.piece_shape = piece.shape.value
        if rotated:
            self.piece_shape = rotate(self.piece_shape)
        
        x, y = np.indices(BOARD)
        self.piece_cubes = ((pos.x <= x) & (x < pos.x + self.piece_shape.x) &
                            (pos.y <= y) & (y < pos.y + self.piece_shape.y))
    
    def __repr__(self):
        return f"{self.piece} {self.pos} {'r' if self.rotated else ''}"
    
    def check(self):
        piece = self.piece
        pos = self.pos
        piece_cubes = self.piece_cubes
        piece_shape = self.piece_shape
        # Is piece available
        if piece not in PIECES - frozenset(self.game.used):
            raise PieceNotAvaliable("Piece not available")
        # Is piece valid given card
        if piece not in self.game.card.value:
            raise PieceNotIncludedInCare("Forbidden piece given current card")
            
        # Is piece out of bounds:
        if any(p < 0 for p in pos):
            raise OutOfBounds("Invalid position (<0)")
        if any(p + s > b for p, s, b in zip(pos, piece_shape, BOARD)):
            raise OutOfBounds("Invalid position (>border)")
        
        # Check all heights under piece: should be all equal to z coord -1
        z = self.pos3.z
        
        if z == 0:
            to_check = [C2(pos.x - 1, pos.y),
                        C2(pos.x, pos.y - 1),
                        C2(pos.x + piece_shape.x, pos.y),
                        C2(pos.x, pos.y + piece_shape.y)]
            for c in to_check:
                try:
                    if self.game.board_cubes[c]:
                        break
                except IndexError:
                    continue
            else:
                raise PieceNotNextToAnotherPiece()
        
        all_z = self.game.board_cubes[piece_cubes]
        if not np.all(all_z == (z)):
            raise PieceNotOnOtherPieces(f"Wrong height (should be all {z}):\n{all_z}")
        
        # Is piece next to similar piece
        # Check 5 directions with board_pieces
        forbidden = {C3(pos.x, pos.y, z - piece_shape.z),
                     C3(pos.x - piece_shape.x, pos.y, z),
                     C3(pos.x + piece_shape.x, pos.y, z),
                     C3(pos.x, pos.y - piece_shape.y, z),
                     C3(pos.x, pos.y + piece_shape.y, z)}

        if any((f, piece_shape) in self.game.board_pieces for f in forbidden):
            raise PieceNextToSimilarPiece()

    def play(self):
        # Add piece_cubes to board_cubes
        self.game.board_cubes[self.piece_cubes] += self.piece_shape.z
        # Add pos & piece to board_pieces 
        self.game.board_pieces[(self.pos3, self.piece_shape)] = self.piece.color
        
        self.game.used.append(self.piece)
        self.game.turn += 1


In [6]:
import random
def random_play():
    g = Game.new(FirstPos.ABC)
    while True:
        try:
            next(g.possible_moves()).play()
        except StopIteration:
            break


In [32]:
%timeit tuple(random_play())

613 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%prun random_play()